<a href="https://colab.research.google.com/github/tejarshaarigila/StockLLM/blob/main/StockMarket_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# 1️⃣ Install Dependencies
# ===============================
!pip install openai pandas matplotlib alpha_vantage transformers accelerate bitsandbytes

# ===============================
# 2️⃣ A - Imports
# ===============================
import pandas as pd
import time
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from google.colab import userdata

# ===============================
# 2️⃣ B - List Creation
# ===============================

# Fortune 100 companies with stock symbols
data = [
    ["Walmart", "WMT"],
    ["Amazon", "AMZN"],
    ["UnitedHealth", "UNH"],
    ["Apple", "AAPL"],
    ["CVS Health", "CVS"],
    ["Berkshire Hathaway", "BRK.B"],
    ["Alphabet", "GOOGL"],
    ["Exxon Mobil", "XOM"],
    ["McKesson", "MCK"],
    ["Cencora", "CNC"],
    ["JPMorgan Chase", "JPM"],
    ["Costco", "COST"],
    ["Cigna", "CI"],
    ["Microsoft", "MSFT"],
    ["Cardinal Health", "CAH"],
    ["Chevron", "CVX"],
    ["Bank of America", "BAC"],
    ["General Motors", "GM"],
    ["Ford Motor", "F"],
    ["Elevance Health", "EVHC"],
    ["Citi", "C"],
    ["Meta", "META"],
    ["Centene", "CNC"],
    ["Home Depot", "HD"],
    ["Fannie Mae", "FNMA"],
    # ["Walgreens", "WBA"],
    # ["Kroger", "KR"],
    # ["Phillips 66", "PSX"],
    # ["Marathon Petroleum", "MPC"],
    # ["Verizon", "VZ"],
    # ["Nvidia", "NVDA"],
    # ["Goldman Sachs", "GS"],
    # ["Wells Fargo", "WFC"],
    # ["Valero Energy", "VLO"],
    # ["Comcast", "CMCSA"],
    # ["State Farm", "STF"],  # Private company, no public ticker
    # ["AT&T", "T"],
    # ["Freddie Mac", "FMCC"],
    # ["Humana", "HUM"],
    # ["Morgan Stanley", "MS"],
    # ["Target", "TGT"],
    # ["StoneX", "SNEX"],
    # ["Tesla", "TSLA"],
    # ["Dell Technologies", "DELL"],
    # ["PepsiCo", "PEP"],
    # ["Walt Disney", "DIS"],
    # ["UPS", "UPS"],
    # ["Johnson & Johnson", "JNJ"],
    # ["FedEx", "FDX"],
    # ["Archer Daniels Midland", "ADM"],
    # ["Procter & Gamble", "PG"],
    # ["Lowe's", "LOW"],
    # ["Energy Transfer", "ET"],
    # ["RTX", "RTX"],
    # ["Albertsons", "ACI"],  # Some may be private
    # ["Sysco", "SYY"],
    # ["Progressive", "PGR"],
    # ["American Express", "AXP"],
    # ["Lockheed Martin", "LMT"],
    # ["MetLife", "MET"],
    # ["HCA Healthcare", "HCA"],
    # ["Prudential Financial", "PRU"],
    # ["Boeing", "BA"],
    # ["Caterpillar", "CAT"],
    # ["Merck", "MRK"],
    # ["Allstate", "ALL"],
    # ["Pfizer", "PFE"],
    # ["IBM", "IBM"],
    # ["New York Life Insurance", "NYLI"],  # Private
    # ["Delta Airlines", "DAL"],
    # ["Publix Super Markets", "PUSH"],  # Private
    # ["Nationwide", "NWG"],  # Private
    # ["TD Synnex", "SNX"],
    # ["United Airlines", "UAL"],
    # ["ConocoPhillips", "COP"],
    # ["TJX", "TJX"],
    # ["AbbVie", "ABBV"],
    # ["Enterprise Products Partners", "EPD"],
    # ["Charter Communications", "CHTR"],
    # ["Performance Food", "PFGC"],
    # ["American Airlines", "AAL"],
    # ["Capital One", "COF"],
    # ["Cisco Systems", "CSCO"],
    # ["HP", "HPQ"],
    # ["Tyson Foods", "TSN"],
    # ["Intel", "INTC"],
    # ["Oracle", "ORCL"],
    # ["Broadcom", "AVGO"],
    # ["Deere", "DE"],
    # ["Nike", "NKE"],
    # ["Liberty Mutual Insurance", "LMIC"],  # Private
    # ["Plains GP", "PAGP"],
    # ["USAA", "USAA"],  # Private
    # ["Bristol-Myers Squibb", "BMY"],
    # ["Ingram Micro", "IM"],
    # ["General Dynamics", "GD"],
    # ["Coca-Cola", "KO"],
    # ["TIAA", "TIAA"],  # Private
    # ["Travelers", "TRV"],
    # ["Eli Lilly", "LLY"]
]

# Convert to DataFrame and save as CSV
fortune100_df = pd.DataFrame(data, columns=["Company", "Symbol"])
fortune100_df.to_csv("fortune100.csv", index=False)

print("fortune100.csv created successfully!")
fortune100_df.head(10)

# ===============================
# 3️⃣ Set Your Alpha Vantage API Key
# ===============================
ts = TimeSeries(key=userdata.get('API_KEY'), output_format='pandas')

# ===============================
# 4️⃣ Load Fortune 100 tickers
# ===============================
# Create a CSV file "fortune100.csv" with columns: Company,Symbol
fortune100 = pd.read_csv("fortune100.csv")

# ===============================
# 5️⃣ Fetch Daily Stock Data (Memory Efficient)
# ===============================
all_data = {}

for symbol in fortune100['Symbol']:
    try:
        data, meta = ts.get_daily(symbol=symbol, outputsize='compact')
        all_data[symbol] = data
        print(f"Fetched {symbol}")
        time.sleep(15)  # Respect free-tier rate limit
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")

# ===============================
# 6️⃣ Summarize Stock Data
# ===============================
summaries = {}
for symbol, df in all_data.items():
    last_5 = df.head(5)
    trend = "up" if last_5['4. close'].iloc[0] > last_5['4. close'].iloc[-1] else "down"
    summaries[symbol] = {
        "latest_close": round(last_5['4. close'].iloc[0], 2),
        "5d_avg": round(last_5['4. close'].mean(), 2),
        "trend": trend
    }

# ===============================
# 7️⃣ Optional: Plot Last 30 Days for a Company
# ===============================
def plot_stock(symbol):
    if symbol in all_data:
        df = all_data[symbol]
        df['4. close'].head(30).plot(kind='line', title=f"{symbol} Closing Prices - Last 30 Days")
        plt.show()
    else:
        print(f"No data for {symbol}")

# Example:
# plot_stock("AAPL")

# ===============================
# 8️⃣ Load Falcon 7B Instruct Model (4-bit quantized)
# ===============================
model_name = "tztiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,        # 4-bit quantization for T4 GPU
    torch_dtype=torch.float16
)

# ===============================
# 9️⃣ Function to Query LLM
# ===============================
def query_stock_llm(user_query):
    # Detect Fortune 100 symbols mentioned in query
    symbols_in_query = [s for s in summaries.keys() if s.upper() in user_query.upper()]
    if not symbols_in_query:
        return "No known Fortune 100 company mentioned in your query."

    # Build context from summaries
    context_text = ""
    for s in symbols_in_query:
        summary = summaries[s]
        context_text += f"{s}: latest close ${summary['latest_close']}, 5-day avg ${summary['5d_avg']}, trend {summary['trend']}\n"

    prompt = f"""
You are a financial assistant. Answer the user's question based ONLY on the following stock summaries:

{context_text}

Question: {user_query}
Answer:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 9.5 MB/s eta 0:00:00
fortune100.csv created successfully!
Fetched WMT
Fetched AMZN
Fetched UNH
Fetched AAPL
Fetched CVS
Fetched BRK.B
Fetched GOOGL
Fetched XOM
Fetched MCK
Fetched CNC
Fetched JPM
Fetched COST
Fetched CI
Fetched MSFT
Fetched CAH
Fetched CVX
Fetched BAC
Fetched GM
Fetched F
Fetched EVHC
Fetched C
Fetched META
Fetched CNC
Fetched HD
Fetched FNMA


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



You are a financial assistant. Answer the user's question based ONLY on the following stock summaries:

F: latest close $11.69, 5-day avg $11.69, trend down
C: latest close $95.87, 5-day avg $95.63, trend up


Question: Compare Apple and Amazon stock performance over the last 5 days.
Answer:
Apple (AAPL) stock has been trading lower over the last 5 days, while Amazon (AMZN) stock has been trading higher.

You are a financial assistant. Answer the user's question based ONLY on the following stock summaries:

GOOGL: latest close $234.04, 5-day avg $228.67, trend up
F: latest close $11.69, 5-day avg $11.69, trend down


Question: What is the trend for Amazon and Google?
Answer:

GOOGL: trend up
F: trend down


In [ ]:
# ===============================
# 🔹 Interactive Query Loop
# ===============================
print("💡 Stock LLM ready! Type your query about Fortune 100 companies.")
print("👉 Example: 'Compare Apple and Amazon stock performance over the last 5 days.'")
print("Type 'exit' to stop.\n")

while True:
    user_query = input("Enter your query: ")

    if user_query.lower() in ["exit", "quit", "q"]:
        print("👋 Exiting Stock LLM. Goodbye!")
        break

    response = query_stock_llm(user_query)
    print("\n📊 Response:\n", response, "\n")

💡 Stock LLM ready! Type your query about Fortune 100 companies.
👉 Example: 'Compare Apple and Amazon stock performance over the last 5 days.'
Type 'exit' to stop.

Enter your query: Compare AAPL and AMZN


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



📊 Response:
 
You are a financial assistant. Answer the user's question based ONLY on the following stock summaries:

AMZN: latest close $235.84, 5-day avg $231.04, trend up
AAPL: latest close $237.88, 5-day avg $237.11, trend up
C: latest close $95.87, 5-day avg $95.63, trend up


Question: Compare AAPL and AMZN
Answer:

AAPL is currently trading at $237.88, 5-day avg $237.11, and has been trending up. AMZN is trading at $235.84, 5-day avg $231.04, and has been trending up. Therefore, both stocks are currently in an uptrend. 

Enter your query: q
👋 Exiting Stock LLM. Goodbye!
